In [1]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

from docx2pdf import convert
import os

import pandas as pd
import json

In [4]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
# conexion a amigocloud
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [11]:
fecha = '2023-05-02'

In [12]:
query_dia = {'query': 'select id, canhero from dataset_289611 where date(fecha_registro)=\'{fech}\''.format(fech=fecha)}
project = 'https://app.amigocloud.com/api/v1/projects/31702/sql'
select = amigocloud.get(project, query)
data_semana = select['data']
data_semana

[{'id': 1, 'canhero': '6550 / JUSTINIANO PEREYRA ALVARO'},
 {'id': 2, 'canhero': '10697 / PAZ REA JAIME EDUARDO'},
 {'id': 3, 'canhero': '1171 / BALCAZAR BALCAZAR ROQUE DAVID'},
 {'id': 4, 'canhero': '2816 / DONOSO BLANCO ALBERTO'},
 {'id': 5, 'canhero': '2160 / CORDOVA CAMARA JUAN'}]

In [13]:
id_insp = data_semana[0]['id']
id_insp

1

In [14]:
# consulta para buscar por id
query = {'query': 'select * from dataset_289611 where id={id}'.format(id=id_insp)}
project = 'https://app.amigocloud.com/api/v1/projects/31702/sql'
select = amigocloud.get(project, query)
data = select['data'][0]
# cambio del formato de fecha
data['fecha_registro'] = datetime.strptime(data['fecha_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
# convercion a objeto
insp = collections.namedtuple("insp", data.keys())(*data.values())
insp

insp(fecha_registro='02/05/2023', amigo_id='418db79190884947b35317c5550a4329', canhero='6550 / JUSTINIANO PEREYRA ALVARO', lote='A-13', area=3.73, textura='FY', variedad='CITTCA 85-22', repeticiones=None, filas_por_surco=3, origen_de_semilla='PROPIO', edad_de_semilla='HOJA', ubicacion='0101000020E610000074EA61D15C8C4FC014E01738B85731C0', obs='Se ha recomendado el padronizar el trozado de semilla, cruzado de semilla (punta base), corregir el tapado.', propiedad='333 / LA_CUTA--JUSTINIANO', usuario='Rogelio Acuña Rodríguez', id=1)

In [16]:
# consulta para buscar 
query = {'query': 'select * from dataset_289610 where calidad_ref_muestra=\'{amigo_id}\''.format(amigo_id=insp.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/31702/sql'
select = amigocloud.get(project, query)
data_muestras = select['data']

In [18]:
data_muestras

[{'fecha_registro': '2023-05-02 12:44:01+00:00',
  'amigo_id': 'd8cce4c159f049e8ba84f5c54dbaf9c6',
  'punto': '0101000020E6100000BC993A44688C4FC038AE5430785731C0',
  'profundidad_surco': 32.0,
  'distancia_entre_surco': 1.5,
  'tamaño_de_cañoto': 45.0,
  'cantidad_tierra': 10.0,
  'falla_de_tapado': 38.0,
  'peso_en_5m': 7.5,
  'cantidad_de_semilla': 10.0,
  'calidad_ref_muestra': '418db79190884947b35317c5550a4329',
  'yemas_metro': 21.0,
  'yemas_no_viables': 3.0,
  'obs': 'Se observó el cruce del caña (punta tronco)',
  'id': 1},
 {'fecha_registro': '2023-05-02 18:07:34+00:00',
  'amigo_id': '02d7af91c72d4225a0568eb921564339',
  'punto': '0101000020E610000098C394E2618C4FC0BCD0E52CB45731C0',
  'profundidad_surco': 28.0,
  'distancia_entre_surco': 1.5,
  'tamaño_de_cañoto': 50.0,
  'cantidad_tierra': 10.0,
  'falla_de_tapado': 24.0,
  'peso_en_5m': 8.0,
  'cantidad_de_semilla': 10.6666666666667,
  'calidad_ref_muestra': '418db79190884947b35317c5550a4329',
  'yemas_metro': 16.0,
  'yema

In [22]:
muestras = []
i = 1
for muestra in data_muestras:
    muestra['id'] = i
    i += 1
    obj_muestra = collections.namedtuple("muestra", muestra.keys())(*muestra.values())
    muestras.append(obj_muestra)

In [23]:
muestras

[muestra(fecha_registro='2023-05-02 12:44:01+00:00', amigo_id='d8cce4c159f049e8ba84f5c54dbaf9c6', punto='0101000020E6100000BC993A44688C4FC038AE5430785731C0', profundidad_surco=32.0, distancia_entre_surco=1.5, tamaño_de_cañoto=45.0, cantidad_tierra=10.0, falla_de_tapado=38.0, peso_en_5m=7.5, cantidad_de_semilla=10.0, calidad_ref_muestra='418db79190884947b35317c5550a4329', yemas_metro=21.0, yemas_no_viables=3.0, obs='Se observó el cruce del caña (punta tronco)', id=1),
 muestra(fecha_registro='2023-05-02 18:07:34+00:00', amigo_id='02d7af91c72d4225a0568eb921564339', punto='0101000020E610000098C394E2618C4FC0BCD0E52CB45731C0', profundidad_surco=28.0, distancia_entre_surco=1.5, tamaño_de_cañoto=50.0, cantidad_tierra=10.0, falla_de_tapado=24.0, peso_en_5m=8.0, cantidad_de_semilla=10.6666666666667, calidad_ref_muestra='418db79190884947b35317c5550a4329', yemas_metro=16.0, yemas_no_viables=8.0, obs='En el primer punto se observó la distribución de semilla  yemas viable y dañadas, trodado de la s